# Elasticsearch Quick Start

<a target="_blank" href="https://colab.research.google.com/github/elasticsearch-labs/colab-notebooks-examples/blob/notebooks-guides/search/00-quick-start.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to the very basics of getting started with simple Elasticsearch queries, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
We'll run through getting the client up and running, indexing a small data set into Elasticsearch, and performing basic searches against your data.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Select **Create deployment**

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [ ]:
!pip install -qU elasticsearch sentence-transformers==2.2.2

# Setup the Embedding Model
For this example, we're using the all-MiniLM-L6-v2, part of the sentence_transformers library. You can read more about this model in [hugging face](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [ ]:
from elasticsearch import Elasticsearch
from getpass import getpass

CLOUD_ID = getpass("Elastic Cloud ID")
CLOUD_PASSWORD = getpass("Elastic Password")

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", CLOUD_PASSWORD)
)

If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally

Confirm that the client has connected with this test.

In [ ]:
print(client.info())

## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of books with the following fields:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create index

Let's create an Elasticsearch index with the correct mappings for our test data.

In [ ]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": "true",
                "similarity": "cosine"
            }
        }
    }
}

# Create the index
client.indices.create(index='book_index', body=mapping)


### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books from this [dataset](https://raw.githubusercontent.com/joemcelroy/elasticsearch-labs/notebooks-guides/colab-notebooks-examples/search/data.json).

In [ ]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/joemcelroy/elasticsearch-labs/notebooks-guides/colab-notebooks-examples/search/data.json"
response = urlopen(url)
books = json.loads(response.read())

actions = []
for book in books:
    actions.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    actions.append(book)
client.bulk(index="book_index", operations=actions)


## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [26]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

##Making Queries

Now that we have indexed the books, we want to perform a semantic search for books that similarly match the query. We embed the query and perform a search.

In [27]:
response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100
    }
})

pretty_response(response)


ID: OOlWP4kB-GB5Evg6zHVx
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Score: 0.8075247

ID: NOlWP4kB-GB5Evg6zHVx
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Score: 0.6946182

ID: NelWP4kB-GB5Evg6zHVx
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Score: 0.66179085

ID: MOlWP4kB-GB5Evg6zHVx
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Score: 0.61159486

ID: OelWP4kB-GB5Evg6zHVx
Publication date: 2012-06-27
Title: Introduction to the Theory of Computation
Summary: Introduction to the theory of computation and complexity theory
Score: 0.58697784

ID: N-lWP4kB-GB5Evg6zHVx
Publication date: 2011-05-13
Title: The Clean C

<ipython-input-27-f6998147c0ac>:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={


## Next Steps

In the next guide, we will learn more on how to apply metadata filtering and querying to your search

[Metadata Filtering and Querying](https://)